In [3]:
import json
import gzip
import bz2
import lzma
import glob
from os.path import basename
from  collections import *
from tqdm.notebook import tqdm
import os
import zstandard as zstd

In [4]:
files = glob.glob('/shared/2/datasets/reddit-dump-all/RC/*.zst')
files.extend(glob.glob('/shared/2/datasets/reddit-dump-all/RC/*.xz'))
files.extend(glob.glob('/shared/2/datasets/reddit-dump-all/RC/*.bz2'))
files.extend(glob.glob('/shared/2/datasets/reddit-dump-all/RS/*.bz2'))
files.extend(glob.glob('/shared/2/datasets/reddit-dump-all/RS/*.xz'))

In [5]:
month_to_files = defaultdict(list)
for fname in files:
    name = basename(fname)
    year_month = name.split('.')[0][3:]
    month_to_files[year_month].append(fname)
print(len(month_to_files))

200


In [6]:
months = sorted(list(month_to_files.keys()), reverse=True)

In [7]:
output_dir = '/shared/0/projects/reddit-political-affiliation/data/bipartite-networks/'

def get_file_type(fname):
    if fname.endswith('bz2'):
        return bz2
    elif fname.endswith('xz'):
        return lzma
    else:
        return zstd
    
#for month, mfiles in tqdm(month_to_files.items(), total=len(month_to_files)):
for month in tqdm(months):
    mfiles = month_to_files[month]    
    user_to_subreddit_counts = defaultdict(Counter)
    for fname in mfiles:
        print(fname)

        ftype = get_file_type(fname)
        try:
            with ftype.open(fname, 'rt') as f:
                for line in f:
                    try:
                        action = json.loads(line) # post or comment
                        subreddit = action['subreddit']
                        user = action['author']
                        user_to_subreddit_counts[user][subreddit] += 1
                    except KeyError as e:
                        #print(repr(e))
                        pass
        except BaseException as e:
            print(repr(e))
    with open(output_dir + month + '.tsv', 'wt') as outf:
        for user, scs in user_to_subreddit_counts.items():
            for sub, count in scs.items():
                outf.write(user + '\t' + sub + '\t' + str(count) + '\n')

/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-12.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-11.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-10.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-09.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-08.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-07.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-06.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-05.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-04.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-03.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-02.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2010-01.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2009-12.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2009-11.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2009-10.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2009-09.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2009-08.xz
/shared/2/datasets/reddit-dump-all/RS/RS_v2_2009-07.xz
/shared/2/

KeyboardInterrupt: 